In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

C=1e+03
gamma=1e-05
clf = svm.SVC(C=C, gamma=gamma, kernel='rbf', probability=True)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Clean data by dropping NaN values in each set.

Training data:

In [ ]:
# Read and quick clean of training data
aug_train = "../input/hr-analytics-job-change-of-data-scientists/aug_train.csv"
train = pd.read_csv(aug_train)
train_df = train.dropna()
test_df = train.tail(100)
train_df.columns[:14] # Shows 14 attributs we have

Test data:

In [ ]:
# Read and quick clean of test data
#aug_test = "../input/hr-analytics-job-change-of-data-scientists/aug_test.csv"
#test = pd.read_csv(aug_test)
#test_df = test.dropna()
#test_df.info() # Displays dtypes of attributes

Graphical representation of training data attributes to total number of candidates grouped by target attribute value using histograms.

In [ ]:
# City
fig = px.histogram(train_df.dropna(), x = "city", color = "target")
fig.show()

In [ ]:
# City development index (CDI)
fig = px.histogram(train_df.dropna(), x = "city_development_index", color = "target")
fig.show()

In [ ]:
# Gender
fig = px.histogram(train_df.dropna(), x = "gender", color = "target")
fig.show()

In [ ]:
# Relevent Experience
fig = px.histogram(train_df.dropna(), x = "relevent_experience", color = "target")
fig.show()

In [ ]:
# Enrolled University
fig = px.histogram(train_df.dropna(), x = "enrolled_university", color = "target")
fig.show()

In [ ]:
# Education Level
fig = px.histogram(train_df.dropna(), x = "education_level", color = "target")
fig.show()

In [ ]:
# Major
fig = px.histogram(train_df.dropna(), x = "major_discipline", color = "target")
fig.show()

In [ ]:
# Experience
fig = px.histogram(train_df.dropna(), x = "experience", color = "target")
fig.show()

In [ ]:
# Code taken from StackOverflow thread: 
# https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = preprocessing.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
    
# Stores categorical attribute columns as names as a list      
string_attributes = train_df.select_dtypes(['object']).columns.to_list()

# use MultiColumnLabelEncoder class to choose columns to encode
# No need to worry about missing data since it has already been handled
train_df_transform = MultiColumnLabelEncoder(columns = string_attributes).fit_transform(train_df)
test_df_transform = MultiColumnLabelEncoder(columns = string_attributes).fit_transform(test_df)

# Display updated dataframe
display(train_df_transform)
display(test_df_transform)

In [ ]:
# Use transformed dataframes to predict
clf = clf.fit(train_df_transform.iloc[:,0:13], train_df_transform.iloc[:,13])
y_pred = clf.predict(test_df_transform.iloc[:,0:13]) 
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix #let's see how good we did
y_true = test_df_transform.iloc[:,13]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
tn, fp, fn, tp 